# システム統合5（用例ベース対話システム）
それでは、これまでに実装してきたモジュールを統合して音声対話システムを作成してみましょう。

- 仕様
    - 音声認識：Google（ストリーミング、オリジナルVAD）
    - 音声合成：Google
    - 対話モデル：用例ベース

In [1]:
# 必要なラブラリを読み込む
# from __future__ import division

# import re
# import sys, os

# import numpy as np
# import math
# import struct

# import pyaudio
# from six.moves import queue

# from pydub import AudioSegment
# from pydub.playback import play

# from google.cloud import speech

# import MeCab

# 実装したライブラリを読み込む
from asr_google_streaming_vad import GoogleStreamingASR, MicrophoneStream
from tts_google import GoogleTextToSpeech
from example_based import ExampleBased

C:\Users\inoue\anaconda3\envs\ai-kouza\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# 音声認識クラスのパラメータ
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

# 音声合成の初期化
tts = GoogleTextToSpeech()

# 対話モデルの初期化
example_based = ExampleBased()

Load from ./data/example-base-data.csv
こんにちは -> こんにちは
趣味は何ですか -> 趣味はスポーツ観戦です
好きな食べ物は何ですか -> りんごです
一番安い商品は何ですか -> 一番安いのはもやしです
最近食べた料理は何ですか -> 最近食べたのはラーメンです
出身はどこですか -> 出身は京都です
印象に残っている旅行はなんですか -> 印象に残っているのはヨーロッパ旅行です
おはよう -> おはようございます
こんばんは -> こんばんは
さようなら -> さようなら
好きな芸能人は誰ですか -> 好きな芸能人はタモリです
好きなスポーツは何ですか -> 好きなスポーツはサッカーです
好きな動物は何ですか -> 好きな動物は犬です
休日は何をされていますか -> 休日は主に散歩しています
仕事は何をしていますか -> 仕事は受付係をしています
最近はまっていることは何ですか -> 最近は映画鑑賞にはまっています
好きな映画は何ですか -> 好きな映画はスターウォーズです
好きなゲームは何ですか -> 好きなゲームはポケモンです
好きなポケモンは何ですか -> 好きなポケモンはピカチュウです
好きな本は何ですか -> 好きな本は純粋理性批判です
好きな小説は何ですか -> 好きな小説は指輪物語です
好きな漫画は何ですか -> 好きな漫画はドラえもんです
好きなアニメは何ですか -> 好きなアニメはドラゴンボールです
おすすめのお店はどこですか -> おすすめのお店はサイゼリアです
得意な料理は何ですか -> チャーハンが得意料理です
好きな教科は何ですか -> 好きな教科は数学です
おすすめの観光地はどこですか -> おすすめは清水寺です
おすすめのお土産は何ですか -> おすすめは八つ橋です
好きなボードゲームは何ですか -> カタンです
好きな数字は何ですか -> 好きな数字は1です
印象に残っている映画は何ですか -> 印象に残っているのはシャイニングです
好きな季節は何ですか -> 好きな季節は夏です
嫌いな食べ物は何ですか -> 嫌いな食べ物はキウイです
嫌いな動物は何ですか -> 嫌いな動物はカラスです
何歳ですか -> 20歳です
健康にはどのように気を付けていますか -> 毎日運動するようにしています

In [3]:
# ユーザ発話に「終了」含まれるまで繰り返す
while(True):

    # 音声認識入力を得る	
    micStream = MicrophoneStream(RATE, CHUNK)
    asrStream = GoogleStreamingASR(RATE, micStream)
    print('<<<please speak>>>')
    result_asr = asrStream.get_asr_result()

    if hasattr(result_asr, 'alternatives') == False:
        print('Invalid ASR input')
        continue

    result_asr_utterance = result_asr.alternatives[0].transcript
    print("ユーザ： " + result_asr_utterance)
    
    # ユーザ発話に「終了」含まれていれば終了
    if "終了" in result_asr_utterance:
        print('<<<終了します>>>')
        break
        
    # 用例を検索（word2vec版を使用）
    result_asr_utterance_mecab = example_based.parse_mecab(result_asr_utterance)
    response, cos_dist_max  = example_based.matching_word2vec(result_asr_utterance_mecab)
    print("%s (%f)" % (response, cos_dist_max))
    
    # システム応答を再生
    tts.generate(response)
    print("システム： " + response)
    tts.play()
    
    print()

<<<please speak>>>
音声パワー： 52.8[dB] 途中結果: 今日はユーザ： こんにちは
こんにちは (1.000000)
システム： こんにちは

<<<please speak>>>
音声パワー： 49.6[dB] 途中結果: 趣味は何ですかユーザ： 趣味は何ですか
趣味はスポーツ観戦です (1.000000)
システム： 趣味はスポーツ観戦です

<<<please speak>>>
音声パワー： 48.2[dB] 途中結果: 最近面白かったことは何ですかユーザ： 最近面白かったことは何ですか
最近は映画鑑賞にはまっています (0.876461)
システム： 最近は映画鑑賞にはまっています

<<<please speak>>>
音声パワー： 48.1[dB] 途中結果: 歳は何歳ですかユーザ： 歳は何歳ですか
20歳です (0.956073)
システム： 20歳です

<<<please speak>>>
音声パワー： 51.9[dB] 途中結果: 終了しますユーザ： 終了します
<<<終了します>>>
